In [16]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from pydantic import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader, TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

In [22]:
# os.environ["OPENAI_API_KEY"] = "sk-9f91f7f5e8eb4dfabb71c9df5f72e7d2"
# os.environ["OPENAI_BASE_URL"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"
os.environ["NEO4J_URI"] = "neo4j://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "pixiu5331"

graph = Neo4jGraph()


In [24]:
# Read the wikipedia article
raw_documents = TextLoader("edu.txt").load()

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)


In [26]:
# llm = ChatOpenAI(
#     api_key="sk-9f91f7f5e8eb4dfabb71c9df5f72e7d2",
#     base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
#     model="deepseek-v3",
# )

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
)
llm_transformer = LLMGraphTransformer(llm=llm)

# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(documents)
# Store to neo4j
graph.add_graph_documents(
    graph_documents, 
    baseEntityLabel=True, 
    include_source=True
)

KeyboardInterrupt: 